In [2]:
import numpy as np

import openvino as ov

In [5]:
#ov_core = ov.Core()
#ov_model = ov_core.read_model('resnet50.xml')
#compiled_model = ov.compile_model(ov_model, 'CPU')

compiled_model = ov.compile_model('resnet50.xml', device_name='CPU')

In [6]:
dummy_input = np.zeros((1, 3, 224, 224))
print(dummy_input.shape)

(1, 3, 224, 224)


In [8]:
result = compiled_model.infer_new_request(dummy_input)
print(result)

{<ConstOutput: names[x.45] shape[?,1000] type: f32>: array([[-2.93597654e-02,  4.32350725e-01, -5.84305301e-02,
        -2.91103542e-01, -2.55336672e-01,  5.13132036e-01,
        -1.02258757e-01, -3.52007240e-01, -5.48715666e-02,
        -3.16291958e-01, -1.39626577e-01, -3.27999979e-01,
        -1.80215269e-01, -1.08302385e-01, -5.65252662e-01,
        -3.47736388e-01, -3.35917950e-01, -4.26620036e-01,
        -4.87623960e-01, -2.44593531e-01, -1.67377844e-01,
         9.06900913e-02, -7.02377111e-02, -1.42605364e-01,
        -3.86343777e-01, -1.87442899e-01,  5.16163647e-01,
         1.65662751e-01, -4.46559414e-02,  8.70012581e-01,
        -3.28092843e-01,  3.04786205e-01,  2.11769611e-01,
        -1.73410878e-01,  2.47580826e-01,  8.68984759e-02,
        -1.78922545e-02, -2.42940709e-02,  6.93321645e-01,
        -2.42273211e-02, -6.44435138e-02,  2.53315717e-01,
         2.84774572e-01, -1.09304845e-01,  2.29991358e-02,
         1.52209491e-01, -1.03446096e-03, -1.98426455e-01,
   

In [9]:
print(compiled_model.outputs)
print(compiled_model.outputs[0].get_any_name())
print(result['x.45'].shape)
print(result[0].shape)

[<ConstOutput: names[x.45] shape[?,1000] type: f32>]
x.45
(1, 1000)
(1, 1000)


In [10]:
pred = result['x.45'][0]
print(pred[:50])

[-0.02935977  0.43235072 -0.05843053 -0.29110354 -0.25533667  0.51313204
 -0.10225876 -0.35200724 -0.05487157 -0.31629196 -0.13962658 -0.32799998
 -0.18021527 -0.10830238 -0.56525266 -0.3477364  -0.33591795 -0.42662004
 -0.48762396 -0.24459353 -0.16737784  0.09069009 -0.07023771 -0.14260536
 -0.38634378 -0.1874429   0.51616365  0.16566275 -0.04465594  0.8700126
 -0.32809284  0.3047862   0.21176961 -0.17341088  0.24758083  0.08689848
 -0.01789225 -0.02429407  0.69332165 -0.02422732 -0.06444351  0.25331572
  0.28477457 -0.10930485  0.02299914  0.15220949 -0.00103446 -0.19842646
 -0.03066348 -0.11292056]
